# Cortes
>$n$: Quantidade de tippos de itens  
>$m$: Quantidade de padrões de corte  

## Variáveis
>$x_j$: Quantidade que será utilizada do padrão de corte $j$, para $j = 1,\ldots, m$  
>$s_j = 
  \begin{cases}
    1 : \text{Caso o padrão } j \text{ seja utilizado} \\
    0 : \text{Caso não seja}
  \end{cases}
$

## Parâmetros
>$D_i$: Demanda do item $i$, para $i = 1,\ldots, n$  
>$P_{ij}$: Padrão de corte $i$ do tipo de item $j$, para $i = 1,\ldots, n$ e $j = 1,\ldots, m$  
>$R$: Máximo permitido de padrões de corte  

## Modelo  

\begin{align*}
    \hbox{min} \ \ 
        & \sum_{j=1}^m x_i \\
        
    \hbox{s.a.} \ \ 
        & \sum_{j=1}^m P_{ij} x_j \geq D_i && i = 1, \ldots, n \\
        & \sum_{j=1}^m s_j \leq R                              \\
        & x_j \leq M s_j                   && j = 1, \ldots, m \\
        & M = \sum_{i=1}^n D_i                                 \\
        & s_j \in \{0, 1\}                 && j = 1, \ldots, m \\
        & x_{j} \in +\Z                    && j = 1, \ldots, m \\
\end{align*}

In [1]:
from itertools import product


def cut_patterns(M: int, C: list[int]) -> list[list[int]]:
    corte_minimo = M - min(C)
    cortes = product(*(range(M//i + 1) for i in C))
    x = [i for i in cortes if corte_minimo < sum(a*b for a, b in zip(i, C)) <= M]

    return x

In [2]:
import pandas as pd
from IPython.display import display

def create_table(M: int, C: list[int]):
    patterns = cut_patterns(M, C)
    df = pd.DataFrame(patterns, columns=C)
    df["Resto"] = M - df.mul(C).sum(axis=1)
    # display(df)
    
    return df.to_string()

In [5]:
from pymprog import *


def modelo_cortes(M, R, C, D):
    P = [*zip(*cut_patterns(M, C))]

    n = len(D)
    m = len(P[0])

    begin("cortes")

    x = var("x", m, int)
    s = var("s", m, bool)

    minimize( sum(x) )

    for i in range(n):
        sum(P[i][j] * x[j] for j in range(m)) >= D[i]

    M = sum(D)
    for j in range(m):
        x[j] <= s[j] * M

    sum(s) <= R

    solver(int, tm_lim= 120_000)
    solve()

    obj = vobj()

    end()

    return obj, x, P

In [6]:
import os

for file in os.listdir("./input/"):
    with open(f"input/{file}", "r") as f:
        M = int(f.readline())

        R = int(f.readline())

        C, D = [], []
        for i in f.readlines():
            c, d = map(int, i.split())
            C.append(c)
            D.append(d)

        print(f"\n{file:=^75}")

        # create_table(M, C)
        obj, varx, P = modelo_cortes(M, R, C, D)
        P = [*zip(*P)]

        with open(f"output_b/{file}", "w") as f:
            f.write(create_table(M, C) + "\n\n")

            f.write(f"Tamanho da bobina matriz: {M}\n")
            f.write(f"Tamanho das bobinas menores: {C}\n")
            f.write(f"Demandas: {D}\n\n")

            f.write(f"Quantidade de bobinas usadas: {int(obj)}\n\n")

            desperdicio_total = 0

            for i, x in enumerate(varx):
                if x.primal:
                    resto = M - sum(a*b for a, b in zip(P[i], C))
                    desperdicio_total += int(x.primal) * resto

                    f.write(f"Padrão {i:>2}: {P[i]} resto {resto}\n")
                    f.write(f"Quantidade usada: {int(x.primal)}\n")
                    f.write(f"Quantidade desperdiçada: {int(x.primal) * resto}\n")
                    f.write(f"Quantidade satisfeita de cada demanda: {tuple(j * int(x.primal) for j in P[i])}\n\n")

            f.write(f"Quantidade total de papel gasto: {int(obj) * M}\n")
            f.write(f"Quantidade desperdiçada: {desperdicio_total}\n")


==============================inst_200_5.txt===============================
GLPK Simplex Optimizer 5.0
57 rows, 102 columns, 282 non-zeros
      0: obj =   0.000000000e+00 inf =   1.209e+04 (5)
     13: obj =   3.446229167e+03 inf =   0.000e+00 (0)
*    20: obj =   3.338166667e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer 5.0
57 rows, 102 columns, 282 non-zeros
102 integer variables, 51 of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+    20: mip =     not found yet >=              -inf        (1; 0)
Solution found by heuristic: 3478
Solution found by heuristic: 3389
+ 31240: mip =   3.389000000e+03 >=   3.339000000e+03   1.5% (3936; 4593)
+ 34826: >>>>>   3.385000000e+03 >=   3.339000000e+03   1.4% (4038; 5191)
+ 38697: >>>>>   3.340000000e+03 >=   3.339000000e+03 < 0.1% (4041; 6067)
Solution found by heuristic: 3339
+ 47619: mip =   3.339000000e+03 >=     tree is empty   0.0% (0; 25469)
INTEGER OPTIMAL SOLUTION FOUN